In [154]:
import pandas as pd
import random
import dask.dataframe as dd
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np



In [155]:
# ddf = dd.read_csv(timeseries_file_path)
# 
# # Randomly sample 1% of the dataset
# sample_ddf = ddf.sample(frac=0.01)
# 
# # Compute the sample, converting the Dask DataFrame to a Pandas DataFrame
# sample_df = sample_ddf.compute()
# 
# print(sample_df.head())

sample_df = dd.read_csv('/Users/fionaturner-hehlen/Desktop/EnergyDataHackdays2024/100_15_2021.csv').compute()
sample_df.head()

,id,timestamp,value_kwh,year
0,f75148515d279e2c6c36207fa4002faf,2021-05-08T00:15:00.000Z,0.0,2021
1,f75148515d279e2c6c36207fa4002faf,2021-01-30T06:30:00.000Z,0.0,2021
2,f75148515d279e2c6c36207fa4002faf,2021-01-25T23:15:00.000Z,0.0,2021
3,f75148515d279e2c6c36207fa4002faf,2021-02-03T18:30:00.000Z,0.0,2021
4,f75148515d279e2c6c36207fa4002faf,2021-06-17T19:15:00.000Z,0.0,2021


In [156]:
# Add month column
sample_df['timestamp'] = pd.to_datetime(sample_df['timestamp'])
sample_df['year_month'] = sample_df['timestamp'].dt.to_period('M')
sample_df

/var/folders/hh/vz7rsbq10c71_9h873rwpw7w0000gn/T/ipykernel_44801/3963185989.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  sample_df['year_month'] = sample_df['timestamp'].dt.to_period('M')


,id,timestamp,value_kwh,year,year_month
0,f75148515d279e2c6c36207fa4002faf,2021-05-08 00:15:00+00:00,0.000,2021,2021-05
1,f75148515d279e2c6c36207fa4002faf,2021-01-30 06:30:00+00:00,0.000,2021,2021-01
2,f75148515d279e2c6c36207fa4002faf,2021-01-25 23:15:00+00:00,0.000,2021,2021-01
3,f75148515d279e2c6c36207fa4002faf,2021-02-03 18:30:00+00:00,0.000,2021,2021-02
4,f75148515d279e2c6c36207fa4002faf,2021-06-17 19:15:00+00:00,0.000,2021,2021-06
...,...,...,...,...,...
1168001,c33ffe928e1b305d0df6ca2639664949,2021-09-05 17:30:00+00:00,0.019,2021,2021-09
1168002,c33ffe928e1b305d0df6ca2639664949,2021-03-17 12:45:00+00:00,0.020,2021,2021-03
1168003,c33ffe928e1b305d0df6ca2639664949,2021-04-10 05:15:00+00:00,0.007,2021,2021-04
1168004,c33ffe928e1b305d0df6ca2639664949,2021-04-18 17:30:00+00:00,0.006,2021,2021-04


In [157]:
sample_df_sorted = sample_df.sort_values(['id', 'year_month', 'timestamp'])
sample_df_sorted

,id,timestamp,value_kwh,year,year_month
277418,035293b7f95ace18fc6c574cc4e6cc74,2021-01-01 00:00:00+00:00,0.026,2021,2021-01
137121,035293b7f95ace18fc6c574cc4e6cc74,2021-01-01 00:15:00+00:00,0.047,2021,2021-01
270796,035293b7f95ace18fc6c574cc4e6cc74,2021-01-01 00:30:00+00:00,0.036,2021,2021-01
603670,035293b7f95ace18fc6c574cc4e6cc74,2021-01-01 00:45:00+00:00,0.065,2021,2021-01
686382,035293b7f95ace18fc6c574cc4e6cc74,2021-01-01 01:00:00+00:00,0.030,2021,2021-01
...,...,...,...,...,...
629793,ff010cb3b00651e7e6ef2788f7904c08,2021-12-31 22:45:00+00:00,0.010,2021,2021-12
1050075,ff010cb3b00651e7e6ef2788f7904c08,2021-12-31 23:00:00+00:00,0.009,2021,2021-12
634643,ff010cb3b00651e7e6ef2788f7904c08,2021-12-31 23:15:00+00:00,0.010,2021,2021-12
885376,ff010cb3b00651e7e6ef2788f7904c08,2021-12-31 23:30:00+00:00,0.469,2021,2021-12


In [158]:
time_delta_hours = 1
time_delta = time_delta_hours * 4 # amount of 15 minute intervals

In [159]:
df_derivative = sample_df_sorted
df_derivative['last_value'] = df_derivative.groupby(['id', 'year_month'])['value_kwh'].shift(int(time_delta))
df_derivative['demand_change_kwh'] = df_derivative['value_kwh'] - df_derivative['last_value']
df_derivative.fillna(0, inplace=True)

df_derivative

,id,timestamp,value_kwh,year,year_month,last_value,demand_change_kwh
277418,035293b7f95ace18fc6c574cc4e6cc74,2021-01-01 00:00:00+00:00,0.026,2021,2021-01,0.000,0.000
137121,035293b7f95ace18fc6c574cc4e6cc74,2021-01-01 00:15:00+00:00,0.047,2021,2021-01,0.000,0.000
270796,035293b7f95ace18fc6c574cc4e6cc74,2021-01-01 00:30:00+00:00,0.036,2021,2021-01,0.000,0.000
603670,035293b7f95ace18fc6c574cc4e6cc74,2021-01-01 00:45:00+00:00,0.065,2021,2021-01,0.000,0.000
686382,035293b7f95ace18fc6c574cc4e6cc74,2021-01-01 01:00:00+00:00,0.030,2021,2021-01,0.026,0.004
...,...,...,...,...,...,...,...
629793,ff010cb3b00651e7e6ef2788f7904c08,2021-12-31 22:45:00+00:00,0.010,2021,2021-12,0.009,0.001
1050075,ff010cb3b00651e7e6ef2788f7904c08,2021-12-31 23:00:00+00:00,0.009,2021,2021-12,0.278,-0.269
634643,ff010cb3b00651e7e6ef2788f7904c08,2021-12-31 23:15:00+00:00,0.010,2021,2021-12,0.257,-0.247
885376,ff010cb3b00651e7e6ef2788f7904c08,2021-12-31 23:30:00+00:00,0.469,2021,2021-12,0.009,0.460


In [160]:
# def apodize(signal, window_type='hamming', alpha=2.5):
#     """
#     Apply apodization (windowing) to a signal.
# 
#     Parameters:
#         signal (np.array): The input signal to be apodized.
#         window_type (str): The type of window function to apply.
#                            Options: 'hamming', 'hanning', 'blackman', 'gaussian', 'kaiser'.
#         alpha (float): Shape parameter for the Gaussian or Kaiser window (if applicable).
# 
#     Returns:
#         np.array: The apodized signal.
#     """
# 
#     N = len(signal)  # Length of the signal
# 
#     # Select the window type
#     if window_type == 'hamming':
#         window = np.hamming(N)
#     elif window_type == 'hanning':
#         window = np.hanning(N)
#     elif window_type == 'blackman':
#         window = np.blackman(N)
#     elif window_type == 'gaussian':
#         window = np.exp(-0.5 * ((np.arange(N) - N/2) / (alpha * N/2))**2)
#     elif window_type == 'kaiser':
#         window = np.kaiser(N, alpha)
#     else:
#         raise ValueError(f"Unsupported window type: {window_type}")
# 
#     # Apply the window to the signal
#     apodized_signal = signal * window
# 
#     return apodized_signal


In [161]:
#def max_apodize(signal, window_type='hamming', alpha=0.05):
#    return apodize(signal, window_type=window_type, alpha=alpha).max()

In [164]:
#def min_apodize(signal, window_type='hamming', alpha=0.05):
#    return apodize(signal, window_type=window_type, alpha=alpha).min()

In [165]:
rolling_average = 5

In [166]:
def smooth_signal(signal, window_size=rolling_average):
    """
    Smooth the input signal using a rolling mean (moving average).
    
    Parameters:
        signal (np.array): The input signal to be smoothed.
        window_size (int): The size of the rolling window (must be odd).
    
    Returns:
        np.array: The smoothed signal.
    """
    if window_size % 2 == 0:
        raise ValueError("Window size must be odd to keep symmetry.")
    
    # Convert to pandas Series to use rolling mean
    signal_series = pd.Series(signal)
    
    # Apply rolling mean
    smoothed_signal = signal_series.rolling(window=window_size, center=True, min_periods=1).mean().to_numpy()
    
    return smoothed_signal

In [167]:
def smooth_max(signal, window_size=rolling_average):
    return smooth_signal(signal, window_size).max()

def smooth_min(signal, window_size=rolling_average):
    return smooth_signal(signal, window_size).min()

In [168]:
from scipy.signal import savgol_filter

savgol_order = 3

def max_savgol(signal, window_size=rolling_average, polyorder=savgol_order):
    return savgol_filter(signal, window_size, polyorder).max()

def min_savgol(signal, window_size=rolling_average, polyorder=savgol_order):
    return savgol_filter(signal, window_size, polyorder).min()


In [169]:
# Step 1: Group by 'id' and extract features
agg_funcs = {
    'value_kwh': ['mean', 'std', 'min', 'max'],
    'demand_change_kwh': ['mean', 'std', 'max', smooth_max, max_savgol, 'min', smooth_min, min_savgol]
}


monthly_features_df = df_derivative.groupby(['id', 'year_month']).agg(agg_funcs)

monthly_features_df

value_kwh                          \
                                                 mean       std    min    max   
id                               year_month                                     
035293b7f95ace18fc6c574cc4e6cc74 2021-01     0.050968  0.018580  0.000  0.095   
                                 2021-02     0.046885  0.019188  0.000  0.097   
                                 2021-03     0.045167  0.018714  0.000  0.095   
                                 2021-04     0.042099  0.018532  0.000  0.096   
                                 2021-05     0.037923  0.018846  0.000  0.095   
...                                               ...       ...    ...    ...   
ff010cb3b00651e7e6ef2788f7904c08 2021-08     0.021886  0.112757  0.002  0.909   
                                 2021-09     0.022301  0.114516  0.002  0.904   
                                 2021-10     0.094271  0.185862  0.003  0.917   
                                 2021-11     0.135227  0.216254  0.005  0.907   
                                 2021-12     0.154849  0.243728  0.009  2.143   

                                            demand_change_kwh            \
                                                         mean       std   
id                               year_month                               
035293b7f95ace18fc6c574cc4e6cc74 2021-01        -7.728495e-06  0.023369   
                                 2021-02         2.604167e-05  0.022901   
                                 2021-03        -1.276882e-05  0.022326   
                                 2021-04        -6.944444e-07  0.022818   
                                 2021-05        -1.814516e-05  0.023058   
...                                                       ...       ...   
ff010cb3b00651e7e6ef2788f7904c08 2021-08        -3.360215e-07  0.161654   
                                 2021-09         1.156482e-19  0.164209   
                                 2021-10         2.221102e-04  0.276113   
                                 2021-11         8.090278e-05  0.334945   
                                 2021-12        -2.217742e-05  0.340981   

                                                                          \
                                               max smooth_max max_savgol   
id                               year_month                                
035293b7f95ace18fc6c574cc4e6cc74 2021-01     0.072     0.0414   0.056686   
                                 2021-02     0.071     0.0350   0.052657   
                                 2021-03     0.067     0.0348   0.053657   
                                 2021-04     0.076     0.0312   0.054400   
                                 2021-05     0.077     0.0458   0.077229   
...                                            ...        ...        ...   
ff010cb3b00651e7e6ef2788f7904c08 2021-08     0.906     0.4450   0.853429   
                                 2021-09     0.901     0.4348   0.785486   
                                 2021-10     0.907     0.4880   0.785829   
                                 2021-11     0.896     0.4906   0.827086   
                                 2021-12     1.510     1.1750   1.584857   

                                                                          
                                               min smooth_min min_savgol  
id                               year_month                               
035293b7f95ace18fc6c574cc4e6cc74 2021-01    -0.090    -0.0454  -0.077314  
                                 2021-02    -0.081    -0.0558  -0.077371  
                                 2021-03    -0.078    -0.0478  -0.071314  
                                 2021-04    -0.070    -0.0480  -0.065943  
                                 2021-05    -0.074    -0.0490  -0.066257  
...                                            ...        ...        ...  
ff010cb3b00651e7e6ef2788f7904c08 2021-08    -0.906    -0.4450  -0.859857  
                                 2021-09   